# Analisis de superficial del csv de clicks
### En este notebook exploramos los registros del csv con el objetivo de buscar relaciones entre las distintas variables

In [1]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## Leemos el csv, y dropeamos las columnas action_id por estar llena de nans y wifi_connection por estar llena de False

In [2]:
installs = pd.read_csv("../data/installs.csv")
''', dtype={'auction_type_id': 'float64', 'country': 'int64', 'date': 'str',
                                                   'device_id': 'int64', 'platform':'int64', 'ref_type_id': 'int64',
                                                   'source_id':'int64'}, parse_dates = ['date'])'''
installs.drop('click_hash', axis=1, inplace = True)
installs.drop('attributed', axis=1, inplace = True)
installs['created'] = pd.to_datetime(installs['created'], errors = 'coerce')
installs['day'] = installs['created'].dt.date
installs['hour'] = installs['created'].dt.hour
installs.head(5)

D:\Users\Lucas\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (4,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,created,application_id,ref_type,ref_hash,implicit,device_countrycode,device_brand,device_model,session_user_agent,user_agent,event_uuid,kind,wifi,trans_id,ip_address,device_language,day,hour
0,2019-04-24 06:23:29.495,1,1494519392962156891,4716708407362582887,True,6287817205707153877,NaN,3.739127e+17,adjust.com,NaN,79837499-2f2a-4605-a663-e322f759424f,app_open,NaN,NaN,4243443387795468703,3.301378e+18,2019-04-24,6
1,2019-04-24 02:06:01.032,1,1494519392962156891,7143568733100935872,False,6287817205707153877,NaN,7.805539e+18,adjust.com,NaN,NaN,NaN,NaN,NaN,4724288679627032761,3.301378e+18,2019-04-24,2
2,2019-04-20 10:15:36.274,1,1494519392962156891,5230323462636548010,True,6287817205707153877,NaN,8.355496e+18,adjust.com,NaN,dda99e3c-9c4b-487d-891c-79f0a02cb4a8,app_open,NaN,NaN,8291809486355890410,4.060930e+18,2019-04-20,10
3,2019-04-20 21:56:47.151,1,1494519392962156891,5097163995161606833,True,6287817205707153877,NaN,2.355772e+18,adjust.com,NaN,7010c3ce-0fcf-46c6-9be8-374cc0e20af4,app_open,NaN,NaN,4006811922873399949,3.301378e+18,2019-04-20,21
4,2019-04-20 22:40:41.239,1,1494519392962156891,6328027616411983332,False,6287817205707153877,NaN,6.156971e+18,adjust.com,NaN,NaN,NaN,NaN,NaN,3386455054590810771,3.301378e+18,2019-04-20,22


In [3]:
installs=installs.drop(['ref_type','implicit','device_countrycode','device_brand','device_model','session_user_agent','user_agent','event_uuid','kind','wifi','trans_id','ip_address','device_language'],axis=1)

In [4]:
installs['created'].max()

Timestamp('2019-04-26 23:59:58.788000')

In [5]:
installs['created'].min()

Timestamp('2019-04-18 00:00:01.560000')

## Revisamos el tipo de cada columna

In [6]:
installs.dtypes

created           datetime64[ns]
application_id             int64
ref_hash                   int64
day                       object
hour                       int64
dtype: object

## Elegimos las columnas con menor entropia o pocos valores diferentes y nos quedamos con los 5 valores mas comunes

In [7]:
#grupos = []#, 'hour', 'day']
#top5 = {}
#for i in grupos:
#    top5[i]= installs[i].value_counts().head(5).index.tolist()
#top5

In [8]:
installs = installs.sort_values(by = 'created')
installs = installs.sort_values(by = 'ref_hash')
installs['repeticiones'] = 1
device_ids = installs['ref_hash'].to_frame().set_index('ref_hash')
#installs = pd.get_dummies(installs, columns = grupos)
#for column in grupos:
#    for value in top5[column]:
#        installs[column + str(value)] = (installs[column] == value)*1

In [9]:
fecha_minima=pd.to_datetime('2019-04-18 00:00:00')
fecha_minima
fecha_tope=pd.to_datetime('2019-04-21 00:00:00')
installs_ventana1=installs.loc[((installs['created']>fecha_minima) & (installs['created']<fecha_tope) )].copy(deep=False)

In [10]:
fecha_minima=pd.to_datetime('2019-04-19 00:00:00')
fecha_minima
fecha_tope=pd.to_datetime('2019-04-22 00:00:00')
installs_ventana2=installs.loc[((installs['created']>fecha_minima) & (installs['created']<fecha_tope) )].copy(deep=False)

In [11]:
fecha_minima=pd.to_datetime('2019-04-20 00:00:00')
fecha_minima
fecha_tope=pd.to_datetime('2019-04-23 00:00:00')
installs_ventana3=installs.loc[((installs['created']>fecha_minima) & (installs['created']<fecha_tope) )].copy(deep=False)

In [12]:
fecha_minima=pd.to_datetime('2019-04-21 00:00:00')
fecha_minima
fecha_tope=pd.to_datetime('2019-04-24 00:00:00')
installs_ventana4=installs.loc[((installs['created']>fecha_minima) & (installs['created']<fecha_tope) )].copy(deep=False)

In [13]:
fecha_minima=pd.to_datetime('2019-04-24 00:00:00')
fecha_minima
fecha_tope=pd.to_datetime('2019-04-27 00:00:00')
installs_ventana7=installs.loc[((installs['created']>fecha_minima) & (installs['created']<fecha_tope) )].copy(deep=False)

In [14]:
features = ['sum', 'mean', 'std', 'min', 'max']
ventanas = [installs_ventana1, installs_ventana2, installs_ventana3, installs_ventana4, installs_ventana7]
numero_de_ventanas = [1,2,3,4,7]
contador = 0
for ventana in ventanas:
    features_ventana = ventana.groupby('ref_hash').agg(features).fillna(0)
    level0 = features_ventana.columns.get_level_values(0)
    level1 = features_ventana.columns.get_level_values(1)
    features_ventana.columns = level0 + "_" + level1
    features_ventana.to_csv('../xgb/features_installs_lucas_ventana' + str(numero_de_ventanas[contador]) + '.csv')
    contador = contador + 1